# Statistical Hypothesis Testing

We want to test whether a certain hypothesis is likely to be true.

**Hypotheses** are assertions like “this coin is fair” or “data scientists prefer Python to R” or “people are more likely to navigate away from the page without ever reading the content if we pop up an  advertisement that can be translated into statistics about data.

Those **statistics** can be thought of **observations** of random variables from known distributions, which allows us to make statements about how likely those assumptions are to hold.

We have a **null hypothesis** $H_{0}$ that represents some default position, and some **alternative hypothesis** $H_{1}$ that we’d like to compare it with.

## Example: Flipping a Coin

We have a coin and we want to test whether it’s fair.

The coin has come probability p of landing heads, and so null hypothesis is that the coin is fair which corresponds to p = 0.5

$H_{0}$: p=0.5

$H_{1}:  p \neq 0.5$

In [1]:
import math
import scipy.stats

def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

def normal_cdf(x, mu=0,sigma=1):    
    return scipy.stats.norm(mu, sigma).cdf(x)


# the normal cdf _is_ the probability the variable is below a threshold
normal_probability_below = normal_cdf

# it's above the threshold if it's not below the threshold
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# it's between if it's less than hi, but not less than lo
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# it's outside if it's not between
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)



We can also do the reverse—find either the nontail region or the (symmetric) interval around the mean that accounts for a certain level of likelihood. For example, if we want to find an interval centered at the mean and containing 60% probability, then we find the cutoffs where the upper and lower tails each contain 20% of the probability (leaving 60%):

In [2]:
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    #return scipy.stats.norm(mu, sigma).pdf(p)
    return scipy.stats.norm(mu, sigma).ppf(p)

def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds that contain the specified probability"""
    tail_probability = (1 - probability) / 2
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    return lower_bound, upper_bound

Let’s say that we choose to flip the coin n = 1000 times. If our hypothesis of fairness is true, X should be distributed approximately normally with mean 500 and standard deviation 15.8:

In [3]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

In [4]:
print("mu=", mu_0, "sigma",sigma_0)

mu= 500.0 sigma 15.811388300841896


How willing we are to make a **type 1 error** (“false positive”), in which we reject H 0 even though it’s true. 
For reasons lost to the annals of history, this willingness is often set at 5% or 1%. Let’s choose 5%.

Consider the test that rejects $H_{0}$ if X falls outside the bounds given by:

In [5]:
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [6]:
print("Bounds ", lo, hi)

Bounds  469.010248385 530.989751615


In [7]:
normal_probability_between(lo, hi, mu_0, sigma_0)

0.95000000000000018

There is just a 5% chance we observe an X that lies outside this interval, which is the exact significance we wanted.

If $H_{0}$ is true, then, approximately 19 times out of 20, this test will give the correct result.

We are also often interested in **the power of a test**, which is the probability of not making a type 2 error, in which we fail to reject $H_{0}$ even though it’s false.

Let’s check what happens if p is really 0.55, so that the coin is slightly biased toward heads.

In [8]:
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print("Low ", lo, " hi ", hi)

Low  469.010248385  hi  530.989751615


In [9]:
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

print("Mu ", mu_1, " sigma ", sigma_1)

Mu  550.0  sigma  15.732132722552274


In [10]:
# a type 2 error means we fail to reject the null hypothesis. Which will happen when X is still 
# in our original interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability

print("power ", power)

power  0.886547781098


Imagine instead that our null hypothesis was that the coin is not biased toward heads, or that p ≤ 0.5. 
In that case we want a one-sided test that rejects the null hypothesis when X is much larger than 50 but not when X is smaller than 50.
So a 5% significance test involves using normal_probability_below to find the cutoff below which 95% of the probability lies:


In [11]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
print("hi ", hi)
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability

print("Power ", power)

hi  526.007419394
Power  0.936378997858


This is a **more powerful test**, since it no longer rejects $H_{0}$ when X is below 469 (which is very unlikely 
to happen if $H_{1}$ is true) and instead rejects $H_{0}$ when X is between 526 and 531

An alternative way of thinking about the preceding test involves **p-values**. Instead of choosing bounds based on some probability cutoff, we compute the probability—assuming $H_{0}$ is true—that we would see a value at least as extreme as the one we actually observed.

In [12]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2 * normal_probability_above(x, mu, sigma)

    # if x is less than the mean, the tail is what's less than x
    return 2 * normal_probability_below(x, mu, sigma)

If we were to see 530 heads, we would compute:

In [13]:
print(two_sided_p_value(529.5, mu_0, sigma_0))

0.062077215796


One way to convince yourself that this is a sensible estimate is with a **simulation**:

In [14]:
import random
extreme_value_count = 0

for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0 for _ in range(1000))
    
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1
    
print(extreme_value_count / 100000)
        

0.06005


Since the p-value is greater than our 5% significance, we don’t reject the null hypothesys. If we instead saw 532 heads, the p-value would be:

In [15]:
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

# Confidence interval


A third approach is to construct a confidence interval around the observed value of the parameter.
For example, we can estimate the probability of the unfair coin by looking at the average value of the Bernoulli variables corresponding to each flip—1 if heads, 0 if tails. If we observe 525 heads out of 1,000 flips, then we estimate p equals 0.525.

How confident can we be about this estimate? Well, if we knew the exact value of p, the central limit theorem tells us that the average of those Bernoulli variables should be approximately normal, with mean p and standard deviation:

```
math.sqrt(p * (1 - p) / 1000)
```
Here we don’t know p, so instead we use our estimate:


In [19]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1-p_hat) / 1000)

Using the normal approximation, we conclude that we are **“95% confident”** that the following interval contains the true parameter p:

In [21]:
lo, hi = normal_two_sided_bounds(0.95, mu, sigma)

print("Confidence interval [{0} {1}]".format(lo, hi))

Confidence interval [0.4940490098153452 0.5559509901846548]


If instead we’d seen 540 heads, then we’d have:

In [24]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print(normal_two_sided_bounds(0.95, mu, sigma))

(0.50910957476724517, 0.57089042523275491)


Here, “fair coin” doesn’t lie in the confidence interval. (The “fair coin” hypothesis doesn’t pass a test that you’d expect it to pass 95% of the time if it were true.)

# P-hacking

A procedure that erroneously rejects the null hypothesis only 5% of the time will—by definition—5% of the time erroneously reject the null hypothesis:

In [27]:
def run_experiment():
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)

experiments = [run_experiment() for _ in range(1000)]

num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])

print(num_rejections)

46


# Example: Running an A/B Test

Try to get people to click on advertisements. One of your advertisers has developed a new energy drink targeted at data scientists, and the VP of Advertisements wants your help choosing between advertisement A (“tastes great!”) and advertisement B (“less bias!”).

You decide to run an experiment by randomly showing site visitorsone of the two advertisements and tracking how many people click on each one.
Let’s say that $N_{A}$ people see ad A, and that $n_{A}$ of them click it. We can think of each adview as a **Bernoulli** trial where $p_{A}$ is the probability that someone clicks ad A. Then (if $N_{A}$ is large, which it is here) we know that $n_{A}/N_{A}$ is approximately a normal random variable with mean $p_{A}$ and standard deviation $\sigma _{A} = \sqrt{p_{A}(1-p_{A})/N_{A}}$.

Similarly, $n_{B}/N_{B}$ is approximately a normal random variable with mean $p_{B}$ and standard deviation $\sigma _{B} = \sqrt{p_{B}(1-p_{B})/N_{B}}$

In [28]:
def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma


If we assume those two normals are **independent** (which seems reasonable, since the individual Bernoulli trials ought to be), then their difference should also be normal with mean $p_{B} − p_{A}$ and standard deviation $\sqrt{p_{A}^{2} + p_{B}^{2}}$

This means we can test the **null hypothesis** that $p_{A}$ and $p_{B}$ are the same (that is, that $p_{A} − p_{B}$ is zero), using the statistic:

In [29]:
def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    
    return (p_B - p_A) / math.sqrt(sigma_A**2 + sigma_B**2)

if “tastes great” gets 200 clicks out of 1,000 views and “less bias” gets 180
clicks out of 1,000 views, the statistic equals:

In [30]:
z = a_b_test_statistic(1000, 200, 1000, 180)
print(z)

-1.1403464899034472


The probability of seeing such a large difference if the means were actually equal would be:

In [31]:
two_sided_p_value(z)

0.25414197654223603

which is large enough that you can’t conclude there’s much of a difference. On the other hand, if “less bias” only got 150 clicks, we’d have:

In [32]:
z = a_b_test_statistic(1000, 200, 1000, 150)
print(z)
two_sided_p_value(z)

-2.948839123097944


0.0031896997062168583

which means there’s only a 0.003 probability you’d see such a large difference if the ads were equally effective.